In [192]:
import numpy as np
import pandas as pd
import plotly.graph_objects as go
import plotly.graph_objects as go
import plotly.express as px
import os
import plotly.io as plt_io
# Enable this to export as iframe
# plt_io.renderers.default = 'iframe'

In [193]:
# Import and clean the excel file
trello_table = pd.read_excel('./INPUT/vsoc_report.xlsx')
vsoc_fields_dict = {'ISS Ticket ID':'T#',
                          'Status': 'STATUS',
                          'Severity Code': 'SEVERITY',
                          'Create Date': 'TIMESTAMP',
                          'Resolution': 'RESOLUTION',
                          'Assigned To': 'T_ISSUER',
                          'Comments' : 'COMMENT',}

trello_table = trello_table[vsoc_fields_dict]
trello_table.rename(columns= vsoc_fields_dict, inplace=True)
severity_mapper = {'SEV1':'CRITICAL', 'SEV2': 'HIGH', 'SEV3': 'MEDIUM', 'SEV4': 'LOW'}
trello_table = trello_table.replace({'SEVERITY': severity_mapper})
trello_table['TIMESTAMP'] = pd.to_datetime(trello_table['TIMESTAMP'])
trello_table['TIMESTAMP'] = trello_table['TIMESTAMP'].dt.tz_localize('GMT').dt.tz_convert('Asia/Riyadh')
trello_table.start_timestamp = trello_table['TIMESTAMP'].min().strftime('%d/%m/%Y|%H:%M:%S')
trello_table.end_timestamp = trello_table['TIMESTAMP'].max().strftime('%d/%m/%Y|%H:%M:%S')
trello_table.no_days = len(trello_table['TIMESTAMP'].dt.normalize().unique())
trello_table.to_csv('./OUTPUT/trello_table.csv', sep=',')

In [194]:
trello_table = pd.read_csv('./INPUT/trello_board.csv')
trello_fields_dict = {'Card Name':'T#',
                          'Card Description': 'DESC',
                          'Labels': 'LABELS',
                          'List Name': 'STATUS', 
                          'T_CRE_TS':'T_CREATION_TIMESTAMP', 
                          'Card ID': 'T_ACK_TIMESTAMP',
                          'T_RES_TS': 'T_RESOLUTION_TIMESTAMP'}
trello_table = trello_table[trello_fields_dict]
trello_table.rename(columns=trello_fields_dict, inplace=True)
trello_table['T_CREATION_TIMESTAMP'] = pd.to_datetime(trello_table['T_CREATION_TIMESTAMP'])
trello_table['T_RESOLUTION_TIMESTAMP'] = pd.to_datetime(trello_table['T_RESOLUTION_TIMESTAMP'])


trello_table.start_timestamp = trello_table['T_CREATION_TIMESTAMP'].min().strftime('%d/%m/%Y|%H:%M:%S')
trello_table.end_timestamp = trello_table['T_CREATION_TIMESTAMP'].max().strftime('%d/%m/%Y|%H:%M:%S')
trello_table.no_days = len(trello_table['T_CREATION_TIMESTAMP'].dt.normalize().unique())

In [195]:
# Only analyze resolved tickets
trello_table = trello_table[trello_table['STATUS'] == 'RESOLVED']
# display(trello_table)
trello_table = trello_table.sort_values(by=['T#'],ignore_index=True)
# To get the card creation timestamp from it's ID
trello_table['T_ACK_TIMESTAMP'] = [x[:8] for x in trello_table['T_ACK_TIMESTAMP']]
trello_table['T_ACK_TIMESTAMP'] = trello_table['T_ACK_TIMESTAMP'].apply(int, base=16)
trello_table['T_ACK_TIMESTAMP'] = pd.to_datetime(trello_table['T_ACK_TIMESTAMP'],unit='s')
trello_table['T_ACK_TIMESTAMP'] = trello_table['T_ACK_TIMESTAMP'].dt.tz_localize('GMT').dt.tz_convert('Asia/Riyadh')

In [196]:
combined_table = pd.merge(trello_table, trello_table, on='T#')
combined_table.sort_values(by=['T#'],ignore_index=True)
combined_table.index += 1 
display(combined_table)
combined_table.to_csv('./OUTPUT/combined_table.csv')

,T#,STATUS_x,SEVERITY,TIMESTAMP,RESOLUTION,T_ISSUER,COMMENT,DESC,LABELS,STATUS_y,T_CREATION_TIMESTAMP,T_ACK_TIMESTAMP,T_RESOLUTION_TIMESTAMP


In [197]:
# @TODO: time is off by one hour for some reason

# Adjust time to resolve to business hours
def f(x):
    idx = pd.date_range(start=x.T_CREATION_TIMESTAMP, end=x.T_RESOLUTION_TIMESTAMP, freq= bh)
    mask = ~((idx.dayofweek == 0) & (idx.hour <= 7))
    return len(idx[mask])

sip_bh = pd.offsets.CustomBusinessHour(start='08:00', end='16:00', weekmask='Sun Mon Tue Wed Thu')

trello_table['T_CREATION_TIMESTAMP_ADJUSTED'] = trello_table['T_CREATION_TIMESTAMP']  + sip_bh
trello_table['BUSINESS_HOURS_TO_RESOLVE'] = trello_table.apply(f, axis=1)

# @TODO: ack time is wrong. Why?
trello_table['T_ACK_TIMESTAMP_ADJUSTED'] = trello_table['T_ACK_TIMESTAMP']  + sip_bh
trello_table['BUSINESS_HOURS_TO_ACK'] = trello_table.apply(f, axis=1)

mtta = round(trello_table['BUSINESS_HOURS_TO_ACK'].mean(), 2) 
mttr =  round(trello_table['BUSINESS_HOURS_TO_RESOLVE'].mean(), 2) 

print('Mean time to resolve = ', mttr, 'business hours')
print('Mean time to ack = ', mtta, 'business hours')

Mean time to resolve =  19.06 business hours
Mean time to ack =  19.06 business hours


/Users/haythm/miniconda3/lib/python3.9/site-packages/pandas/core/arrays/datetimes.py:741: PerformanceWarning:

Non-vectorized DateOffset being applied to Series or DatetimeIndex

/Users/haythm/miniconda3/lib/python3.9/site-packages/pandas/core/arrays/datetimes.py:741: PerformanceWarning:

Non-vectorized DateOffset being applied to Series or DatetimeIndex



In [206]:
# Mapping tags
# tags_mapper = {'S1':'CRITICAL', 'SEV2': 'HIGH', 'SEV3': 'MEDIUM', 'SEV4': 'LOW'}
trello_table["LABELS"] = trello_table["LABELS"].str.replace('\([^)]*\)', '')
trello_table['LABELS'].str.split(',', n=-1, expand=True)
trello_table = pd.concat([trello_table,trello_table.LABELS.str.get_dummies(sep=',')],1)


/var/folders/66/48j3cq2j25j62nj7q71wdpt00000gn/T/ipykernel_7316/84126661.py:3: FutureWarning:

The default value of regex will change from True to False in a future version.

/var/folders/66/48j3cq2j25j62nj7q71wdpt00000gn/T/ipykernel_7316/84126661.py:5: FutureWarning:

In a future version of pandas all arguments of concat except for the argument 'objs' will be keyword-only



In [207]:
display(trello_table)
trello_table.to_csv('./OUTPUT/trello_table.csv')

,T#,DESC,LABELS,STATUS,T_CREATION_TIMESTAMP,T_ACK_TIMESTAMP,T_RESOLUTION_TIMESTAMP,T_CREATION_TIMESTAMP_ADJUSTED,BUSINESS_HOURS_TO_RESOLVE,T_ACK_TIMESTAMP_ADJUSTED,...,S12:CRE,S1:VECTRA,S3:PA,S4:ASC,C1:DC,C2:EUS,C3:GRC,C4:USERS,C5:SEC,C6:APP
0,SOCP00703028513,VPN login success from suspicious location,"C3:GRC , C4:USERS , R12:IN , S3:PA",RESOLVED,2021-10-31 09:23:00,2021-10-31 11:03:51+03:00,2021-11-08 14:29:00,2021-10-31 10:23:00,54,2021-10-31 12:03:51+03:00,...,0,0,1,0,0,0,1,0,0,0
1,SOCP00703028815,"Suspicious Admin - Mr. Rafeek from DC team, He...","C1:DC , R1:NI-KA , S1:VECTRA",RESOLVED,2021-10-31 14:13:00,2021-10-31 14:24:08+03:00,2021-11-07 16:31:00,2021-10-31 15:13:00,42,2021-10-31 15:24:08+03:00,...,0,1,0,0,1,0,0,0,0,0
2,SOCP00703030162,An alert for outgoing connection to China on p...,"C4:USERS , R9:NI-OT , S3:PA",RESOLVED,2021-11-01 08:56:00,2021-11-01 09:12:43+03:00,2021-11-08 11:19:00,2021-11-01 09:56:00,43,2021-11-01 10:12:43+03:00,...,0,0,1,0,0,0,0,1,0,0
3,SOCP00703031013,User logged in from Bahrain using 2FA. Due to ...,"C4:USERS , R1:NI-KA , S4:ASC",RESOLVED,2021-11-01 19:34:28,2021-11-02 08:57:15+03:00,2021-11-02 13:28:00,2021-11-02 09:00:00,6,2021-11-02 09:57:15+03:00,...,0,0,0,1,0,0,0,1,0,0
4,SOCP00703032009,A duplicate alert for SOCP00703012537. It was ...,"C4:USERS , R9:NI-OT , S4:ASC",RESOLVED,2021-11-02 05:37:00,2021-11-02 08:59:34+03:00,2021-11-02 14:00:00,2021-11-02 09:00:00,7,2021-11-02 09:59:34+03:00,...,0,0,0,1,0,0,0,1,0,0
5,SOCP00703032222,SMB Brute-Force. New laptops – Part of tech re...,"C2:EUS , R1:NI-KA , S1:VECTRA",RESOLVED,2021-11-02 08:35:00,2021-11-02 08:57:04+03:00,2021-11-07 10:19:00,2021-11-02 09:35:00,26,2021-11-02 09:57:04+03:00,...,0,1,0,0,0,1,0,0,0,0
6,SOCP00703034177,Connections attempts from internal APIPA range...,"C5:SEC , O:169.254 , O:NS , R2:FP-TU , S3:PA",RESOLVED,2021-11-03 01:53:00,2021-11-03 10:01:55+03:00,2021-11-08 10:28:00,2021-11-03 09:00:00,27,2021-11-03 11:01:55+03:00,...,0,0,1,0,0,0,0,0,1,0
7,SOCP00703034616,Azure Security Center log source appeared to b...,"C5:SEC , R1:NI-KA , S12:CRE",RESOLVED,2021-11-03 06:28:00,2021-11-03 10:01:43+03:00,2021-11-03 10:04:00,2021-11-03 09:00:00,3,2021-11-03 11:01:43+03:00,...,1,0,0,0,0,0,0,0,1,0
8,SOCP00703035015,SMB Brute-Force. New laptops – Part of tech re...,"C2:EUS , R1:NI-KA , S1:VECTRA",RESOLVED,2021-11-03 10:53:00,2021-11-03 12:39:15+03:00,2021-11-07 16:28:00,2021-11-03 11:53:00,22,2021-11-03 13:39:15+03:00,...,0,1,0,0,0,1,0,0,0,0
9,SOCP00703035322,VPN access from AE. Duplicate of SOCP00703013856.,"C3:GRC , C4:USERS , R12:IN , S3:PA",RESOLVED,2021-11-03 14:38:00,2021-11-04 11:13:07+03:00,2021-11-08 14:43:00,2021-11-03 15:38:00,25,2021-11-04 12:13:07+03:00,...,0,0,1,0,0,0,1,0,0,0


In [213]:
# Generate basic bar plot for the main features of si-report
required_fields = ['S1:VECTRA', 'R1:NI-KA']
for required_field in required_fields:
    trello_table_copy = trello_table.copy()
    series = pd.value_counts(trello_table_copy[required_field])
    mask = (series/series.sum() * 100).lt(1.0)
    trello_table_copy[required_field] = np.where(trello_table_copy[required_field].isin(series[mask].index),'OTHERS ≤ 1.0%'.format(required_field),trello_table_copy[required_field])
    required_field_count = trello_table_copy[required_field].value_counts()    
    required_field_count = required_field_count.rename_axis(required_field).reset_index(name='{}_COUNT'.format(required_field))
    required_field_count = required_field_count.reset_index(drop=True)
    required_field_count.index.rename('NO.', inplace=True)
    required_field_count.index+=1
    required_field_count['{}_PCT'.format(required_field)] = required_field_count['{}_COUNT'.format(required_field)] / required_field_count['{}_COUNT'.format(required_field)].sum()
    required_field_count.to_csv('./OUTPUT/{}.csv'.format(required_field), sep=',')
    my_layout = go.Layout(title_text='Distribution of Common Event<br>From {} to {}<br>sorted by {} '.format(
        trello_table.start_timestamp, trello_table.end_timestamp,required_field), font=dict(color='#7f7f7f',size=10))
    
    my_data = go.Bar(x=required_field_count['{}_COUNT'.format(required_field)], y=required_field_count[required_field], orientation='h')
    
    fig = go.Figure(data=my_data, layout=my_layout)
    # tune the font_size to your needs
    fig.update_layout(font_size=10,title_x=0.5)
    fig.update_traces(
        marker_color='rgb(231,198,91)',
        marker_line_color='black',
        marker_line_width=1.0,
        textposition='outside',
        opacity=1.0)
    fig.write_image('./OUTPUT/{}_COUNT.pdf'.format(required_field))
    # fig.show()

KeyError: 'S1:VECTRA'

In [200]:
# Generate basic bar plot for the main features of si-report
required_fields = ['T_ISSUER', 'SEVERITY', 'RESOLUTION']
for required_field in required_fields:
    trello_table_copy = trello_table.copy()
    series = pd.value_counts(trello_table_copy[required_field])
    mask = (series/series.sum() * 100).lt(1.0)
    trello_table_copy[required_field] = np.where(trello_table_copy[required_field].isin(series[mask].index),'OTHERS ≤ 1.0%'.format(required_field),trello_table_copy[required_field])
    required_field_count = trello_table_copy[required_field].value_counts()    
    required_field_count = required_field_count.rename_axis(required_field).reset_index(name='{}_COUNT'.format(required_field))
    required_field_count = required_field_count.reset_index(drop=True)
    required_field_count.index.rename('NO.', inplace=True)
    required_field_count.index+=1
    required_field_count['{}_PCT'.format(required_field)] = required_field_count['{}_COUNT'.format(required_field)] / required_field_count['{}_COUNT'.format(required_field)].sum()
    required_field_count.to_csv('./OUTPUT/{}.csv'.format(required_field), sep=',')
    my_layout = go.Layout(title_text='Distribution of Common Event<br>From {} to {}<br>sorted by {} '.format(
        trello_table.start_timestamp, trello_table.end_timestamp,required_field), font=dict(color='#7f7f7f',size=10))
    
    my_data = go.Bar(x=required_field_count['{}_COUNT'.format(required_field)], y=required_field_count[required_field], orientation='h')
    
    fig = go.Figure(data=my_data, layout=my_layout)
    # tune the font_size to your needs
    fig.update_layout(font_size=10,title_x=0.5)
    fig.update_traces(
        marker_color='rgb(231,198,91)',
        marker_line_color='black',
        marker_line_width=1.0,
        textposition='outside',
        opacity=1.0)
    fig.write_image('./OUTPUT/{}_COUNT.pdf'.format(required_field))
    # fig.show()

In [201]:
# Generate a summary table of the main features
summary_table = trello_table.groupby(['STATUS', 'SEVERITY', 'RESOLUTION']).size().unstack(fill_value=0)
summary_table.to_csv('./OUTPUT/SUMMARY_TABLE.csv', sep=',')

In [208]:
tickets_count = trello_table['T#'].value_counts()
tickets_count = tickets_count.rename_axis('T#').reset_index(name='T#_COUNT')
tickets_count = tickets_count.reset_index(drop=True)
tickets_count.index.rename('NO.', inplace=True)
tickets_count.index+=1
tickets_count['T#_PCT'] = tickets_count['T#_COUNT'] / tickets_count['T#_COUNT'].sum()
s = pd.to_datetime(trello_table['T_CREATION_TIMESTAMP'])
tickets_count = s.groupby(s.dt.floor('d')).size().reset_index(name='COUNT')
# Plot ----------------------------------------------------------------------------------------------
my_layout = go.Layout(
    title='Global Font',
    font=dict(color='#7f7f7f', size=10),
    showlegend=False)
my_data = go.Scatter(
    x=tickets_count['T_CREATION_TIMESTAMP'],
    y=tickets_count['COUNT'],
    mode='lines+markers+text')
fig = go.Figure(data=my_data, layout=my_layout)

fig.update_layout(shapes=[
    # Line Horizontal
    go.layout.Shape(
        type='line',
        x0=tickets_count['T_CREATION_TIMESTAMP'].min(),
        y0=tickets_count['COUNT'].mean(),
        x1=tickets_count['T_CREATION_TIMESTAMP'].max(),
        y1=tickets_count['COUNT'].mean(),
        line=dict(
            color='black',
            width=2,
            dash='longdash',
        ),
    ),
])

fig.add_trace(
    go.Scatter(
        x=[
            tickets_count['T_CREATION_TIMESTAMP'].max() -
            pd.Timedelta(days=trello_table
        .no_days) / 2
        ],
        y=[tickets_count['COUNT'].mean()],
        mode='markers+text',
        name='Markers and Text',
        hoverinfo='skip',
        textposition='top right'))

fig.update_traces(
    marker_color='rgb(231,198,91)',
    marker_line_color='black',
    marker_line_width=1,
    opacity=1.0)
fig.update_layout(
    title_text='VSOC tickets trendline grouped by the day<br>From {} to {}. dashed line '
               'represents<br>the average no. VSOC tickets ({} '
               'VSOC tickets per day)'.format(trello_table
            .start_timestamp, trello_table
            .end_timestamp,
                                             int(tickets_count['COUNT'].mean())),font_size=10,title_x=0.5)

fig.update_traces(
marker_color='rgb(231,198,91)',
marker_line_color='black',
marker_line_width=1.0,
opacity=1.0)
# tune the font_size to your needs
fig.update_layout(font_size=12)
# fig.show()
fig.write_image('./OUTPUT/TRENDLINE.pdf')

In [210]:
s = pd.to_datetime(trello_table['T_CREATION_TIMESTAMP'])
tickets_count = s.groupby(s.dt.floor('H')).size().reset_index(name='COUNT')
tickets_count['DAY'] = tickets_count['T_CREATION_TIMESTAMP'].dt.day_name()
tickets_count['HR'] = tickets_count['T_CREATION_TIMESTAMP'].dt.hour
heat_map_data = [go.Heatmap(x=tickets_count['DAY'], y = tickets_count['HR'], z = tickets_count['COUNT'],colorscale='Viridis')]
heat_map_layout = go.Layout(title_text='Heatmap of no. VSOC tickets created per hour of the day<br>From {} '
                                       'to {}.'.format(trello_table.start_timestamp, trello_table.end_timestamp,))
fig = go.Figure(data = heat_map_data, layout = heat_map_layout)
fig.show()
fig.write_image('./OUTPUT/HEATMAP.pdf')

In [204]:
#  FIGURE 1: EOI Chart - OJA
ce_categories_center = ['caused by external actors', 'caused by internal technical actors', 'caused by internal non-technical actors']
last_month_ce_count = [33, 35, 12]
current_month_ce_count = [19, 22, 15]
ce_sum = list(((np.array(current_month_ce_count ) + np.array(last_month_ce_count))))
delta_ce = list(((np.array(current_month_ce_count) - np.array(last_month_ce_count )) / last_month_ce_count ) * 100)
delta_ce = np.round(delta_ce, decimals=1)

green_tag = '<span style="color:green">▼</span>'
red_tag = '<span style="color:red">▲</span>'
ibm_logo_path = os.path.join(os.getcwd(), 'INPUT/sip-logo.png')

fig = go.Figure()
fig.add_trace(go.Bar(
    x=ce_categories_center,
    y=last_month_ce_count,
    name='SI count on August',
    marker_color='rgb(231,198,91)', 
    marker_line_color='black',
    textposition='inside', 
    text=last_month_ce_count,
    marker_line_width=1.5, 
    opacity=1.0, width=0.5))

fig.add_trace(go.Bar(
    x=ce_categories_center,
    y=current_month_ce_count,
    name='SI count in September', 
    text=current_month_ce_count,
    textposition='inside',
    marker_color='white', 
    marker_line_color='black',
    marker_line_width=1.5, 
    opacity=0.6, width=0.6))

fig.add_trace(go.Bar(
    x=ce_categories_center, 
    y=max(current_month_ce_count,last_month_ce_count),
    text=delta_ce,
    textposition='outside',
    marker_line_width=1.5, 
    opacity=0, 
    width=0.6,
    showlegend=False))

for i, (x, y, z) in enumerate(zip(current_month_ce_count, last_month_ce_count, delta_ce)):
    print(i, x, y, z)
    if z > 0:
        fig.add_annotation(x=i,y=max(x+1, y+1),text='<b>▲+{}%</b>'.format(z),showarrow=False,font=dict(color='red', size=12),align='center')
    else:
        fig.add_annotation(x=i,y=max(x+1, y+1),text='<b>▼{}%</b>'.format(z),showarrow=False,font=dict(color='green', size=12),align='center')


title_plot1 = '<b> Increase {} and decrease {} in security incidents count categorized by root cause</b>'.format(red_tag, green_tag)

fig.layout.images = [dict(
    source=ibm_logo_path,
    xref='paper', 
    yref='paper',
    x=0.05, 
    y=1.05,
    sizex=0.11, 
    sizey=0.15,
    xanchor='center', 
    yanchor='bottom')]

fig.update_layout(
    height=600,
    width=1200,
    template='plotly_white',
    font_family='IBM Plex Sans',
    font_size=14,
    barmode='overlay', 
    xaxis_tickangle=0,
    title=title_plot1,
    title_x=0.5, 
    font=dict(color='black'),
    legend=dict(
    orientation='h',
    yanchor='top',
    y=1.1,
    xanchor='center',
    x=0.5))
    
fig.show()
fig.write_image('./OUTPUT/EOI.pdf')

0 19 33 -42.4
1 22 35 -37.1
2 15 12 25.0


In [205]:
last_month_ce_count = [4, 14, 12, 25, 2, 3]
current_month_ce_count = [2, 5, 2, 9, 5, 7]
# delta_count_ce = last_
delta_percentage_ce = list(((np.array(current_month_ce_count) - np.array(last_month_ce_count )) / np.array(last_month_ce_count)) * 100)
delta_percentage_ce = np.round(delta_percentage_ce, decimals=1)
ce_sum = current_month_ce_count  + last_month_ce_count

print('last_month_sum: {}'.format(np.average(last_month_ce_count)))
print('current_month_ce_sum: {}'.format(np.average(current_month_ce_count)))
print('delta_ce_sum: {}'.format(np.average(delta_ce)))

last_month_sum: 10.0
current_month_ce_sum: 5.0
delta_ce_sum: -18.166666666666668
